# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [771]:
import pandas as pd
import numpy as np

DATA_FOLDER = '../../ADA2017-Tutorials/02 - Intro to Pandas/Data' # Use the data folder provided in Tutorial 02 - Intro to Pandas.
PATH_SEPARATOR = '/'

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

The goal here is to go through each folders in the ebola folder and for each one, concatenate all the CSV into a single dataframe.
We need to join the file, but, as we don't have the same column in every files, we need to merge/fills columns from one file to another.
This is exactly what the append method of a DataFrame does, it merges columns and fill empty cell with NaN.

At the end of this operation, we'll be in possession of a single DataFrame for the whole contries.
Another choice we made was to make `date` and `Description`, `Variable`, `variable` column as indexes. This is a detail, but it simplifies the sum over the columns (indexes are not part of columns), and the `.loc` method is more readable when addressing indexes.

The next step was to filter the interessting rows with respect to the labels, as they are different in each contry but can also be different from one file to another we need handles more that one label per country.
Once the rows were fileterd with respect to the labels, we need to sum, for each day all the columns (that represents cities in the country), this is done by keeping only columns that are numbers and add them up.

For each country, we can then return an array of data, representing the number of cases each day.

In [795]:
FOLDER = DATA_FOLDER + PATH_SEPARATOR + "ebola"
COUNTRY_FOLDERS = ['guinea_data', 'sl_data', 'liberia_data']
COUNTRY_INDEXES = [['Description', 'Date'], ['variable', 'date'], ['Variable', 'Date']]
COUNTRY_GROUP = ['Date', 'date', 'Date']
COUNTRY_NEW_CASES_LABELS= [['New cases of confirmed'], ['new_confirmed'], ['New case/s (confirmed)']]
COUNTRY_DEATH_LABELS= [['New deaths registered today (confirmed)', 'New deaths registered'], ['etc_new_deaths'], ['Newly reported deaths']]

def compute(country_folder, indexes=[]):
    country_folder = FOLDER + PATH_SEPARATOR + country_folder
    files = listdir(country_folder)

    dfs = [pd.read_csv(country_folder + PATH_SEPARATOR + file, index_col=indexes) for file in files]
    assert(len(dfs) > 0)
    result = dfs[0].append(dfs[1:])

    return result

In [796]:
def average_per_day(df, group):
    assert(df.empty == False)
    df = df.set_index([group])
    df.index = pd.to_datetime(df.index)
    df = df.apply(pd.to_numeric, errors='ignore')

    grouped = df.groupby(pd.TimeGrouper("m"))

    s = grouped[0].mean()
    s.index = s.index.map(lambda x: x.strftime('%Y-%m'))
    return s

In [797]:
def average_for_label(df, group, labels = []):
    sum = pd.DataFrame()
    for label in labels:
        rows = df.loc[label].apply(pd.to_numeric, errors='ignore')
        filtered = rows.select_dtypes(include=[np.number])
        grouped = filtered.sum(axis=1).groupby(group)
        sum = sum.append(grouped.sum().reset_index(), ignore_index=True) # Sum cities first then group by date
    return average_per_day(sum, group)

In [801]:
result = pd.DataFrame()
for i in range(len(COUNTRY_FOLDERS)):
    df = compute(COUNTRY_FOLDERS[i], COUNTRY_INDEXES[i])
    # There are two ways here, either we remove total and sums the colums, or we could only use total, assuming that it's correctly computed
    if 'Totals' in df.columns.values:
        del df['Totals']
        
    if 'National' in df.columns.values:
        del df['National']
    
    new_cases = average_for_label(df, COUNTRY_GROUP[i], COUNTRY_NEW_CASES_LABELS[i])
    deaths = average_for_label(df, COUNTRY_GROUP[i], COUNTRY_DEATH_LABELS[i])
    labels = new_cases.map(lambda x: COUNTRY_FOLDERS[i].split('_')[0])
    
    result = result.append(pd.concat([new_cases, deaths, labels], axis=1))

result.columns = ['Average number of new cases per days', 'Average number of deaths per day', 'Country']
result



,Average number of new cases per days,Average number of deaths per day,Country
2014-08,12.400000,3.200000,guinea
2014-09,12.562500,3.800000,guinea
2014-10,6.000000,15.000000,guinea
2014-08,19.600000,4.100000,sl
2014-09,36.275862,3.481481,sl
2014-10,57.535714,4.142857,sl
2014-11,69.894737,1.500000,sl
2014-12,54.333333,NaN,sl
2014-06,2.142857,1.857143,liberia
2014-07,1.818182,4.272727,liberia


## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [820]:
FOLDER = DATA_FOLDER + PATH_SEPARATOR + "microbiome"
PREFIX = "MID"
NUMBER = 9

result = pd.DataFrame()

for i in range(1, NUMBER+1):
    result = result.append(pd.read_excel(FOLDER + PATH_SEPARATOR + f"MID{i}.xls", sheetname='Sheet 1', header=None))

header = pd.read_excel(FOLDER + PATH_SEPARATOR + "metadata.xls", sheetname='Sheet1', header=None)
    
result.fillna('unknown')
header


,0,1,2
0,BARCODE,GROUP,SAMPLE
1,MID1,EXTRACTION CONTROL,NaN
2,MID2,NEC 1,tissue
3,MID3,Control 1,tissue
4,MID4,NEC 2,tissue
5,MID5,Control 2,tissue
6,MID6,NEC 1,stool
7,MID7,Control 1,stool
8,MID8,NEC 2,stool
9,MID9,Control 2,stool


## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [822]:
from IPython.core.display import HTML
df = pd.read_csv(DATA_FOLDER+'/titanic.xls')
HTML(filename=DATA_FOLDER+'/titanic.html')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd0 in position 0: invalid continuation byte

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [660]:
# Write your answer here